In [39]:
# - Numpy
import numpy as np
import torch
import pickle
import pandas as pd
from torch.nn import CrossEntropyLoss, MSELoss
import torch.nn as nn
# - Matplotlib
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [12, 4]
plt.rcParams['figure.dpi'] = 300

# - Rockpool time-series handling
from rockpool import TSEvent, TSContinuous
import torch
from rockpool.nn.modules import LinearTorch, LIFTorch
from rockpool.parameters import Constant
from rockpool.nn.combinators import Sequential
# - Pretty printing
try:
    from rich import print
except:
    pass

# - Display images
from IPython.display import Image

# - Disable warnings
import warnings
warnings.filterwarnings('ignore')
# - Import the computational modules and combinators required for the networl
from rockpool.nn.modules import LIFTorch, LinearTorch
from rockpool.nn.combinators import Sequential, Residual
from rockpool.transform import quantize_methods as q
from torch.utils.data import Dataset,random_split,DataLoader
from pathlib import Path
from tqdm.asyncio import tqdm
from rockpool.nn.networks.wavesense import WaveSenseNet
from rockpool.devices import xylo as x
from rockpool.nn.networks import SynNet,WaveSenseNet    
from sklearn.metrics import accuracy_score, f1_score, precision_recall_fscore_support, confusion_matrix
from sklearn.metrics import f1_score, precision_recall_fscore_support, confusion_matrix
from sklearn.metrics import f1_score, precision_recall_fscore_support, confusion_matrix
from rockpool.nn.combinators import Sequential, Residual
from rockpool.nn.modules.torch.lif_torch import PeriodicExponential
from rockpool.nn.modules import LIFBitshiftTorch
from function import *
from encode import *
from dataset import *
#---------------------#
#    define network   #
#---------------------#
b = 1
c = 16
step = 10
Nhidden = 120
thr = 0.82
thr_out = 2.06
net = WaveSenseNet(n_channels_in=c, 
                   n_classes=4, 
                   dilations=[2,8,16,32],
                   threshold=thr, 
                   threshold_out=thr_out)
net.load('/home/liruixin/workspace/gait_classification/models/model_first_spike_175.pth')

#---------------------#
#      quantize       #
#---------------------#
g = net.as_graph()
spec = x.vA2.mapper(g, weight_dtype='float', threshold_dtype='float', dash_dtype='float')
quant_spec = spec.copy()
# - Quantize the specification
spec.update(q.global_quantize(**spec))
# - Use rockpool.devices.xylo.config_from_specification
config, is_valid, msg = x.vA2.config_from_specification(**spec)
modSim = x.vA2.XyloSim.from_config(config)
print('网络部署完毕')


print('正在加载数据...')
with open('data/dataset/test_dataset.pkl', 'rb') as file:
    test_dataset = pickle.load(file)
print('正在进行升采样...')
test_dataloader = DataLoader(test_dataset,batch_size=1)
print('升采样完成')

def model_test(test_dataloader, model):
    accuracy = []
    f1s = []
    precision = []
    recall = []
    cmlist= []
    test_preds = []
    test_targets = []
    for batch, target in tqdm(test_dataloader):
        with torch.no_grad():
            batch = (((batch)).clip(0,15).transpose(1, 2))
            batch = batch.to(torch.int32).squeeze().numpy()
            target = target.type(torch.LongTensor).numpy()
            model.reset_state()
            out_model, _, rec = model(batch, record=True)
            out = vmem_with_fire_xylo(torch.tensor(out_model), rec)
            pred = out.argmax(0).detach()
            test_preds.append(pred)
            test_targets.append(target)

    f1 = f1_score(test_targets, test_preds, average="macro")
    _, test_precision, test_recall, _ = precision_recall_fscore_support(
        test_targets, test_preds, labels=np.arange(4)
    )
    test_accuracy = accuracy_score(test_targets, test_preds)
    cm = confusion_matrix(test_targets, test_preds)
    f1s.append(f1)
    precision.append(test_precision)
    recall.append(test_recall)
    accuracy.append(test_accuracy)
    cmlist.append(cm)
    print(f"F1 Score = {f1}")
    print(f"Val Precision = {test_precision}, Recall = {test_recall}")
    print(f"Val Accuracy = {test_accuracy}")
    print("Confusion Matrix:")
    print(cm)
    
    
model_test(test_dataloader, model=modSim)

ValueError: `weights` must match size of input and output nodes. Got torch.Size([16, 32]), expected (16, 16).